In [ ]:
#Imports
from IPython.display import Image
import requests
import json
import numpy as np
import pandas as pd
import urllib.parse
from IPython.display import display, HTML, IFrame

In [ ]:
########################################################################################################################################################
# DEFINIÇÃO DE VARIÁVEIS GLOBAIS
########################################################################################################################################################
main_embed_url='https://query.wikidata.org/embed.html#'
main_url = "https://query.wikidata.org/sparql"
scholarly_embed_url='https://query-scholarly.wikidata.org/embed.html#'
scholarly_url = "https://query-scholarly.wikidata.org/sparql"

In [ ]:
########################################################################################################################################################
# DEFINIÇÃO DE FUNÇÕES
########################################################################################################################################################
def query_wikidata(query, url=main_url):
    params = { "query": query, "format": "json" }
    result = requests.get(url=url, params=params, headers={"User-agent": "Vitrine NeuroMat"})
    data = result.json()
    try:
        return data["results"]["bindings"]
    except:
        raise "No results were returned!"

def wdqs_iframe(query='', baseurl=main_embed_url, width=600, height=500):
    return IFrame(baseurl+urllib.parse.quote(query), width=width, height=height)

def join_with_and(items):
    if not items:
        return ""
    elif len(items) == 1:
        return items[0]
    elif len(items) == 2:
        return f"{items[0]} and {items[1]}"
    else:
        return ", ".join(items[:-1]) + f", e {items[-1]}"

In [ ]:
########################################################################################################################################################
# DEFINIÇÃO DAS CONSULTAS E VARIÁVEIS
########################################################################################################################################################
query_1 = """SELECT (COUNT(DISTINCT ?authors) AS ?num_authors) (COUNT(DISTINCT ?articles) AS ?num_articles) WHERE {
  ?articles wdt:P2860 wd:Q56592766;
            wdt:P361 wd:Q18477654.
  OPTIONAL {
    { ?articles wdt:P50 ?authors. }
    UNION
    { ?articles wdt:P2093 ?authors. }
  }
}
"""

data = query_wikidata(query_1, scholarly_url)
num_authors = int(data[0]['num_authors']['value'])
num_articles = int(data[0]['num_articles']['value'])

query_2 = """
SELECT DISTINCT ?article ?articleLabel WHERE {
  ?article wdt:P921 wd:Q24575409.
  FILTER NOT EXISTS { ?article wdt:P361 wd:Q18477654. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE], pt, pt-br, en, es, fr, de, ja, mul". }
}
"""

data = query_wikidata(query_2, scholarly_url)
publications = [f"<a href='{article['article']['value']}' target='_blank' >{article['articleLabel']['value']}</a>" for article in data]

query_3="""#defaultView:Graph
SELECT DISTINCT ?Artigo (?Nome_do_artigo AS ?ArtigoLabel) ?rgb ?Artigo_citado ?Num_citações
WITH { # Get articles only once. While here, take the max_order of the authors (P50 and P2093)
  SELECT ?Artigo (max(?ordem_p50) AS ?ordem_max_p50) (max(?ordem_p2093) AS ?ordem_max_p2093) WHERE {
    ?Artigo wdt:P361 wd:Q18477654.
    OPTIONAL {?Artigo p:P50 ?autor_p50. ?autor_p50 pq:P1545 ?ordem_p50}
    OPTIONAL {?Artigo p:P2093 ?autor_p2093. ?autor_p2093 pq:P1545 ?ordem_p2093}
  } GROUP BY ?Artigo
} AS %artigos
WITH { # From the max_order gotten earlier, select the maximun between them
  SELECT DISTINCT ?Artigo ?O_max WHERE {
    INCLUDE %artigos.
    BIND(IF(BOUND(?ordem_max_p50),
            IF(BOUND(?ordem_max_p2093),
               IF(?ordem_max_p50 <= ?ordem_max_p2093, STR(?ordem_max_p2093), STR(?ordem_max_p50)),
               STR(?ordem_max_p50)),
            STR(?ordem_max_p2093)) AS ?O_max)
  }
} AS %ordem
WITH { # Now, with the articles in hand, get the first author (assuming here that there is at least one filled)
  # and get, if exists, the name of the second author.
  SELECT DISTINCT ?Artigo ?autor1Label ?autor2Label ?O_max WHERE {
    INCLUDE %ordem.
    {?Artigo p:P50 ?autor_. ?autor_ ps:P50 ?autor1. ?autor_ pq:P1545 "1".}
    UNION
    {?Artigo p:P2093 ?autor_. ?autor_ ps:P2093 ?autor1. ?autor_ pq:P1545 "1".}
    
    OPTIONAL{{?Artigo p:P50 ?autor__. ?autor__ ps:P50 ?autor2. ?autor__ pq:P1545 "2".}
             UNION
             {?Artigo p:P2093 ?autor__. ?autor__ ps:P2093 ?autor2. ?autor__ pq:P1545 "2".}}
    SERVICE wdsubgraph:wikidata_main { # We need to federate this part to get the author(s) name(s) from the main graph
      SERVICE wikibase:label {bd:serviceParam wikibase:language "[AUTO_LANGUAGE],pt-br,pt,en,es,fr,de,ja".
                              ?autor1 rdfs:label ?autor1Label.
                              ?autor2 rdfs:label ?autor2Label.}
    }
    SERVICE wikibase:label {bd:serviceParam wikibase:language "[AUTO_LANGUAGE],pt-br,pt,en,es,fr,de,ja".
                            ?autor1 rdfs:label ?autor1Label.
                            ?autor2 rdfs:label ?autor2Label.}
  }
} AS %autores
WITH { # Get the last name of the authors
  SELECT DISTINCT ?Artigo ?Autor1_nome ?Autor2_nome ?O_max WHERE {
    INCLUDE %autores.
    BIND(REPLACE(STR(?autor1Label),REPLACE(str(?autor1Label),'[^ ]*$',''),'') AS ?Autor1_nome)
    BIND(REPLACE(STR(?autor2Label),REPLACE(str(?autor2Label),'[^ ]*$',''),'') AS ?Autor2_nome)
  }
} AS %nome_autores
WITH {
  SELECT DISTINCT ?Artigo (COUNT(?Artigo_citando) AS ?Num_citações) WHERE {
    INCLUDE %autores.
    OPTIONAL{?Artigo_citando wdt:P2860 ?Artigo. ?Artigo_citando wdt:P361 wd:Q18477654.}
  } GROUP BY ?Artigo
} AS %num_citacoes
WHERE{
  INCLUDE %nome_autores.
  INCLUDE %num_citacoes.
  # Get the date, if it is filled
  OPTIONAL {?Artigo wdt:P577 ?data_aux. BIND(YEAR(?data_aux) AS ?data_)}
  BIND(IF(BOUND(?data_),STR(?data_),"?") AS ?data)
  
  # Now, if theres one or two authors, put their names and the date,
  # if there is more, put put the first author followed by a 'et al'
  BIND(IF(?O_max="1",
          CONCAT(?Autor1_nome, ", ", ?data),
          IF(?O_max="2",
             CONCAT(?Autor1_nome, ", ", ?Autor2_nome, ", ", ?data),
             CONCAT(?Autor1_nome, " et al, ", ?data))) AS ?Nome_do_artigo)
  
  # If the articles cite each other, get this info.
  OPTIONAL{?Artigo wdt:P2860 ?Artigo_citado. ?Artigo_citado wdt:P361 wd:Q18477654.}
  OPTIONAL{?Artigo_citando wdt:P2860 ?Artigo. ?Artigo_citando wdt:P361 wd:Q18477654.}
  
  # Put some colors in the graph
  BIND(IF(?Num_citações>30,
          "31005e",
          IF(?Num_citações>20,
             "1a009c",
             IF(?Num_citações>15,
                "0587e3",
                IF(?Num_citações>10,
                   "05e3af",
                   IF(?Num_citações>5,
                      "7fe305",
                      IF(?Num_citações>1,
                         "e3dc05",
                         IF(?Num_citações=1,
                            "e37b05",
                            "e30505"))))))) AS ?rgb)
}
"""

iframe_citacoes = wdqs_iframe(width=700, height=500, baseurl=scholarly_embed_url, query=query_3)

query_4 = """SELECT (COUNT(DISTINCT ?revista) AS ?num_revistas) WHERE {
  {?artigo wdt:P2860 wd:Q56592766; wdt:P1433 ?revista.}
  UNION
  { SERVICE wdsubgraph:wikidata_main { ?artigo wdt:P2860 wd:Q56592766; wdt:P1433 ?revista.}}
  FILTER(?revista != wd:Q118398)    # arXiv
  FILTER(?revista != wd:Q107624307) # ufscar
  FILTER(?revista != wd:Q107004545) # repositório da produção intelectual USP
  FILTER(?revista != wd:Q19835482)  # BioRxiv
}"""

data = query_wikidata(query_4, scholarly_url)
num_revistas = data[0]['num_revistas']['value']

query_5 = """SELECT (COUNT(DISTINCT ?article) AS ?num_articles) (COUNT(?item) AS ?num_citations) ?num_authors
WITH {
  SELECT DISTINCT ?article WHERE {
    { ?article wdt:P361 wd:Q18477654. }
    UNION
    { SERVICE wdsubgraph:wikidata_main { ?article wdt:P361 wd:Q18477654.} }
  }
} AS %articles
WITH {
  SELECT (COUNT(DISTINCT ?author) AS ?num_authors) WHERE {
    INCLUDE %articles.
    { ?article wdt:P50|wdt:P2093 ?author. }
    UNION
    { SERVICE wdsubgraph:wikidata_main { ?article wdt:P50|wdt:P2093 ?author. } }
  }
} AS %authors
WHERE {
  INCLUDE %articles.
  INCLUDE %authors.
  OPTIONAL { ?item wdt:P2860 ?article. }
} GROUP BY ?num_authors"""

data = query_wikidata(query_5, scholarly_url)
num_articles = data[0]['num_articles']['value']
num_citations = data[0]['num_citations']['value']
num_authors = data[0]['num_authors']['value']

query_6 = """#defaultView:Graph
SELECT DISTINCT ?author1 ?author1Label ?rgb ?author2 ?author2Label
WITH {
  SELECT ?item WHERE {
    { ?item wdt:P361 wd:Q18477654. }
    UNION
    { SERVICE wdsubgraph:wikidata_main { { ?item wdt:P361 wd:Q18477654. } } }
  }
} AS %articles
WITH {
  SELECT ?author1 ?author2 ?item WHERE {
    INCLUDE %articles.
    { ?item wdt:P50 ?author1, ?author2. }
    UNION
    { SERVICE wdsubgraph:wikidata_main { ?item wdt:P50 ?author1, ?author2. } }
    FILTER(?author1 != ?author2)
  }
} AS %authors
WHERE {
  INCLUDE %authors. 
  BIND( IF(?country=wd:Q155, "409900", "e6350d") AS ?rgb)
  SERVICE wdsubgraph:wikidata_main {
    OPTIONAL { { ?author1 p:P1416 ?class. ?class pq:P276 [wdt:P17 ?country] } UNION { ?author1 wdt:P1416 [wdt:P17 ?country] } UNION { ?author1 wdt:P108 ?class. ?class wdt:P17 ?country } }
    BIND( IF(?country=wd:Q155, "409900", "e6350d") AS ?rgb)
    SERVICE wikibase:label {
      bd:serviceParam wikibase:language "[AUTO_LANGUAGE],pt-br,pt,en,es,fr,de,ja".
      ?author1 rdfs:label ?author1Label.
      ?author2 rdfs:label ?author2Label.
    }
  }
}"""

iframe_affiliation = wdqs_iframe(width=700, height=500, baseurl=scholarly_embed_url, query=query_6)

query_7 = """SELECT DISTINCT ?item ?itemLabel ?qualis ?qualisLabel (COUNT(DISTINCT ?article) AS ?num_articles) WHERE {
  ?item p:P9683 ?qualis_p.
  ?qualis_p ps:P9683 ?qualis;
            pq:P585 "2020-01-01T00:00:00Z"^^xsd:dateTime.
  SERVICE wdsubgraph:scholarly_articles { ?article wdt:P361 wd:Q18477654; wdt:P1433 ?item. }
  FILTER(?item NOT IN (wd:Q118398, wd:Q107624307, wd:Q107004545, wd:Q19835482))
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],pt-br,pt,en,es,fr,de,ja". }
} GROUP BY ?item ?itemLabel ?qualis ?qualisLabel ORDER BY DESC(?num_articles)
"""

data = query_wikidata(query_7, main_url)
journal_1, num_articles_1, qualis_1 = data[0]["itemLabel"]["value"], data[0]["num_articles"]["value"], data[0]["qualisLabel"]["value"]
journal_2, num_articles_2, qualis_2 = data[1]["itemLabel"]["value"], data[1]["num_articles"]["value"], data[1]["qualisLabel"]["value"]
journal_3, num_articles_3, qualis_3 = data[2]["itemLabel"]["value"], data[2]["num_articles"]["value"], data[2]["qualisLabel"]["value"]

query_8 = """#defaultView:BarChart
SELECT DISTINCT ?year (COUNT(DISTINCT ?item) AS ?Número_Artigos) ?qualisLabel WHERE {
  { ?item wdt:P361 wd:Q18477654; wdt:P1433 ?journal; wdt:P577 ?date_. }
  UNION
  { SERVICE wdsubgraph:wikidata_main { ?item wdt:P361 wd:Q18477654; wdt:P1433 ?journal; wdt:P577 ?date_. } }
  BIND(STR(YEAR(?date_)) AS ?year)
  SERVICE wdsubgraph:wikidata_main {
    ?journal wdt:P9683 ?qualis. ?qualis rdfs:label ?label_. FILTER(LANG(?label_)="pt-br")
    BIND(IF(wikibase:isSomeValue(?qualis), "Sem Cla", ?label_) AS ?qualisLabel)
  }
  FILTER(STR(?qualisLabel) NOT IN ("B3", "B4", "B5", "C", "NP", "Sem Cla"))
} GROUP BY ?year ?qualisLabel"""

iframe_qualis = wdqs_iframe(width=700, height=500, baseurl=scholarly_embed_url, query=query_8)

query_9 = """#defaultView:BarChart
SELECT ?num_authors (COUNT(?item) AS ?num_articles) ?qualisLabel
WITH {
  SELECT DISTINCT ?item (COUNT(?author) AS ?num_authors_) ?journal WHERE {
    { ?item wdt:P361 wd:Q18477654; wdt:P50 ?author; wdt:P1433 ?journal. OPTIONAL { ?item wdt:P2093 ?author. } }
    UNION
    { SERVICE wdsubgraph:wikidata_main { ?item wdt:P361 wd:Q18477654; wdt:P50 ?author; wdt:P1433 ?journal. OPTIONAL { ?item wdt:P2093 ?author. } } }
  } GROUP BY ?item ?journal
} AS %articles
WHERE {
  INCLUDE %articles.
  BIND(IF(?num_authors_ = 1, "Único(a)",
          IF(?num_authors_  = 2, "Dupla",
             IF(?num_authors_ = 3, "Tripla",
                ">3"))) AS ?num_authors).
  
  SERVICE wdsubgraph:wikidata_main {
    ?journal wdt:P9683 ?qualis. ?qualis rdfs:label ?label_. FILTER(LANG(?label_)="pt-br")
    BIND(IF(wikibase:isSomeValue(?qualis), "Sem Cla", ?label_) AS ?qualisLabel)
  }
  FILTER(STR(?qualisLabel) NOT IN ("B3", "B4", "B5", "C", "NP", "Sem Cla"))
} GROUP BY ?num_authors ?num_articles ?qualisLabel ORDER BY ?qualisLabel"""

iframe_qualis_by_authors = wdqs_iframe(width=700, height=500, baseurl=scholarly_embed_url, query=query_9)

query_10 = """SELECT DISTINCT ?keyword ?keywordLabel (COUNT(DISTINCT ?item) AS ?num_items) WHERE {
  { ?item wdt:P361 wd:Q18477654; wdt:P921 ?keyword. }
  UNION
  { SERVICE wdsubgraph:wikidata_main { ?item wdt:P361 wd:Q18477654; wdt:P921 ?keyword. } }
  SERVICE wdsubgraph:wikidata_main { SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],pt-br,pt,en,es,fr,de,ja". ?keyword rdfs:label ?keywordLabel } }
} GROUP BY ?keyword ?keywordLabel ORDER BY DESC(?num_items) ?keywordLabel
"""
data = query_wikidata(query_10, scholarly_url)
keywords = [item["keywordLabel"]["value"] for item in data]

query_11 = """#defaultView:BarChart 
SELECT DISTINCT ?year (COUNT(?item) AS ?num_articles) ?keyword ?keywordLabel WHERE {
  { ?item wdt:P361 wd:Q18477654; wdt:P921 ?keyword; wdt:P577 ?date_. }
  UNION
  { SERVICE wdsubgraph:scholarly_articles { ?item wdt:P361 wd:Q18477654; wdt:P921 ?keyword; wdt:P577 ?date_. } }
  BIND(STR(YEAR(?date_)) AS ?year)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],pt-br,pt,en,es,fr,de,ja". }
} GROUP BY ?keyword ?keywordLabel ?year"""

iframe_keywords = wdqs_iframe(width=700, height=500, baseurl=main_embed_url, query=query_11)

query_12 = """#defaultView:Map
SELECT ?affiliation ?affiliationLabel ?country ?countryLabel ?coords WITH {
  SELECT DISTINCT ?researcher WHERE {
    { ?item wdt:P361 wd:Q18477654; wdt:P50 ?researcher. }
    UNION
    { SERVICE wdsubgraph:scholarly_articles { ?item wdt:P361 wd:Q18477654; wdt:P50 ?researcher. } }
  }
} AS %researchers
WITH {
  SELECT DISTINCT ?researcher ?affiliation WHERE {
    INCLUDE %researchers.
    ?researcher wdt:P1416|wdt:P108|wdt:P185 ?affiliation.
  }
} AS %affiliation
WHERE {
  INCLUDE %affiliation.
  { ?affiliation wdt:P625 ?coords. }
  UNION
  { FILTER NOT EXISTS { ?affiliation wdt:P625 []. } ?affiliation wdt:P17|wdt:P27 ?country. ?country wdt:P625 ?coords. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],pt-br,pt,en,es,fr,de,ja". }
}"""

iframe_affiliation = wdqs_iframe(width=700, height=500, baseurl=main_embed_url, query=query_12)

query_13 = """SELECT DISTINCT ?country ?countryLabel ?num_institutions WITH {
  SELECT DISTINCT ?researcher WHERE {
    { ?item wdt:P361 wd:Q18477654; wdt:P50 ?researcher. }
    UNION
    { SERVICE wdsubgraph:scholarly_articles { ?item wdt:P361 wd:Q18477654; wdt:P50 ?researcher. } }
  }
} AS %researchers
WITH {
  SELECT (COUNT(DISTINCT ?institution) AS ?num_institutions) WHERE {
    INCLUDE %researchers.
    ?researcher wdt:P1416|wdt:P108 ?institution.
  }
} AS %institutions
WHERE {
  INCLUDE %researchers.
  INCLUDE %institutions.  
  ?researcher wdt:P1416|wdt:P108 [wdt:P17 ?country].
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],pt-br,pt,en,es,fr,de,ja". }
}"""

data = query_wikidata(query_13, main_url)
country_list = [item["countryLabel"]["value"] for item in data]
num_institutions = data[0]["num_institutions"]["value"]
num_countries = len(country_list)

query_14 ="""SELECT ?num_main_researchers ?num_associate_researchers ?num_postdoc_researchers ?num_doctor_researchers ?num_master_researchers WITH {
  SELECT (COUNT(DISTINCT ?researcher) AS ?num_main_researchers) WHERE {
    ?researcher wdt:P463 wd:Q18477654.
    OPTIONAL { ?researcher wdt:P108 wd:Q18477654. }
  }
} AS %main_researchers
WITH {
  SELECT (COUNT(DISTINCT ?researcher) AS ?num_associate_researchers) WHERE {
    ?researcher wdt:P1416  wd:Q18477654.
  }
} AS %associate_researchers
WITH {
  SELECT ?researcher WHERE {
    { ?article wdt:P361 wd:Q18477654; wdt:P50 ?researcher. }
    UNION
    { SERVICE wdsubgraph:scholarly_articles { ?article wdt:P361 wd:Q18477654; wdt:P50 ?researcher. } }
  }
} AS %researchers
WITH {
  SELECT (COUNT(DISTINCT ?researcher) AS ?num_postdoc_researchers) WHERE {
    INCLUDE %researchers.
    VALUES ?post_doc_affiliation { wd:Q18477654 wd:Q835960 }
    ?researcher wdt:P108 ?post_doc_affiliation.
  }
} AS %postdoc_researchers
WITH {
  SELECT (COUNT(DISTINCT ?researcher) AS ?num_doctor_researchers) WHERE {
    INCLUDE %researchers.
    ?researcher wdt:P185 [].
  }
} AS %doctor_researchers
WITH {
  SELECT (COUNT(DISTINCT ?researcher) AS ?num_master_researchers) WHERE {
    INCLUDE %researchers.
    FILTER EXISTS { ?researcher wdt:P463|wdt:P1416 wd:Q18477654. }
    FILTER EXISTS { ?researcher wdt:P185 []. }
  }
} AS %master_researchers
WHERE {
  INCLUDE %main_researchers.
  INCLUDE %associate_researchers.
  INCLUDE %postdoc_researchers.
  INCLUDE %doctor_researchers.  
  INCLUDE %master_researchers.
}"""

data = query_wikidata(query_14, main_url)
num_main_researchers = data[0]["num_main_researchers"]["value"]
num_associate_researchers = data[0]["num_associate_researchers"]["value"]
num_postdoc_researchers = data[0]["num_postdoc_researchers"]["value"]
num_doctor_researchers = data[0]["num_doctor_researchers"]["value"]
num_master_researchers = data[0]["num_master_researchers"]["value"]

query_15 = """#defaultView:BarChart
SELECT distinct ?num_publications (COUNT(?researcher) AS ?num_researchers) ?countryLabel WITH {
  SELECT DISTINCT ?researcher (COUNT(DISTINCT ?article) AS ?num_articles) WHERE {
    { ?article wdt:P361 wd:Q18477654; wdt:P50 ?researcher. }
    UNION
    { SERVICE wdsubgraph:scholarly_articles { ?article wdt:P361 wd:Q18477654; wdt:P50 ?researcher. } }
    ?researcher wdt:P1416 wd:Q18477654.
  } GROUP BY ?researcher
} AS %researchers
WHERE {
  INCLUDE %researchers.
  OPTIONAL {
    ?researcher wdt:P108 [wdt:P17 ?country].
    ?country rdfs:label ?countryLabel_. FILTER(LANG(?countryLabel_)="pt-br")
  }
  BIND(IF(?num_articles < 6, "0-5", IF(?num_articles < 11, "6-10", IF(?num_articles < 16, "11-15", IF(?num_articles < 21, "16-20", IF(?num_articles < 26, "21-25", IF(?num_articles < 31, "26-30", "Mais de 31")))))) AS ?num_publications).
  BIND(IF(BOUND(?countryLabel_), ?countryLabel_, "*Sem país declarado") AS ?countryLabel)
} GROUP BY ?countryLabel ?num_publications"""

iframe_publications = wdqs_iframe(width=700, height=500, baseurl=main_embed_url, query=query_15)

query_16 = """#defaultView:BarChart
SELECT distinct ?countryLabel ?num_articles ?researcherLabel WITH {
  SELECT DISTINCT ?researcher (COUNT(DISTINCT ?article) AS ?num_articles) WHERE {
    { ?article wdt:P361 wd:Q18477654; wdt:P50 ?researcher. }
    UNION
    { SERVICE wdsubgraph:scholarly_articles { ?article wdt:P361 wd:Q18477654; wdt:P50 ?researcher. } }
    ?researcher wdt:P1416 wd:Q18477654.
  } GROUP BY ?researcher
} AS %researchers
WHERE {
  INCLUDE %researchers.
  OPTIONAL {
    ?researcher wdt:P108 [wdt:P17 ?country].
    ?country rdfs:label ?countryLabel_. FILTER(LANG(?countryLabel_)="pt-br")
  }
  ?researcher rdfs:label ?researcherLabel. FILTER(LANG(?researcherLabel)="pt-br")
  BIND(IF(BOUND(?countryLabel_), ?countryLabel_, "*Sem país declarado") AS ?countryLabel)
}"""

iframe_publications_by_author = wdqs_iframe(width=700, height=500, baseurl=main_embed_url, query=query_16)

query_17 = """#defaultView:BarChart
SELECT distinct ?genderLabel (COUNT(DISTINCT ?researcher) AS ?number_researchers) ?genderLabel2 WITH {
  SELECT DISTINCT ?researcher WHERE {
    { ?article wdt:P361 wd:Q18477654; wdt:P50 ?researcher. }
    UNION
    { SERVICE wdsubgraph:scholarly_articles { ?article wdt:P361 wd:Q18477654; wdt:P50 ?researcher. } }
    ?researcher wdt:P1416 wd:Q18477654.
  } GROUP BY ?researcher
} AS %researchers
WHERE {
  INCLUDE %researchers.
  ?researcher wdt:P21 ?gender.
  ?gender rdfs:label ?genderLabel. FILTER(LANG(?genderLabel)="pt-br")
  ?gender rdfs:label ?genderLabel2. FILTER(LANG(?genderLabel2)="pt-br")
} GROUP BY ?genderLabel ?genderLabel2"""

data = query_wikidata(query_17, main_url)
genders = {item["genderLabel"]["value"]:item["number_researchers"]["value"] for item in data}

In [ ]:
########################################################################################################################################################
# DEFINIÇÃO DOS PARÁGRAFOS
########################################################################################################################################################
paragrafo_1 = """<p style='margin-top: 0.5em;'>O artigo <a href='https://www.wikidata.org/wiki/Q56592766' target='_blank'>Infinite Systems of Interacting Chains with Memory of Variable Length—A Stochastic Model for Biological Neural Nets</a>,  escrito por <a href='https://www.wikidata.org/wiki/Q17489997' target='_blank' >Antonio Galves</a> e <a href='https://www.wikidata.org/wiki/Q59267761' target='_blank' >Eva Löcherbach</a> em 2013, constitui a pedra angular do <a href='https://www.wikidata.org/wiki/Q18477654' target='_blank' >Centro de Pesquisa, Inovação e Difusão em Neuromatemática (CEPID NeuroMat)</a>. Este trabalho introduziu um novo modelo para redes neurais e lançou uma nova linha de investigação.</p>"""
paragrafo_2 = f"<p style='margin-top: 0.5em;'>No contexto do CEPID NeuroMat, participaram desta linha {num_authors} pesquisadores que produziram {num_articles} artigos com base no modelo de 2013 ou contribuições posteriores. O modelo fundador do NeuroMat, chamado <a href='https://www.wikidata.org/wiki/Q24575409'>Modelo Galves-Löcherbach</a>, foi também o objeto de análise de publicações não filiadas ao NeuroMat, como {join_with_and(publications)}.</p>"
paragrafo_3 = f"<p style='margin-top: 0.5em;'>A figura embaixo é o grafo de citações internas do NeuroMat a partir do artigo considerado a base da investigação do centro de pesquisa. <a href='https://query-scholarly.wikidata.org/#{urllib.parse.quote(query_3)}' target='_blank'>Experimente a consulta completa!</a></p>"
paragrafo_4 = f"<p style='margin-top: 0.5em;'>O artigo que lançou o <a href='https://www.wikidata.org/wiki/Q24575409'>Modelo Galves-Löcherbach</a> teve impacto em várias áreas do conhecimento. As publicações associadas ao novo modelo deram-se em {num_revistas} revistas acadêmicas, em áreas como probabilidade, estatística, computação e neurobiologia.</p>"
paragrafo_5 = f"""Desde a sua criação em 2013, a equipe de pesquisadores do CEPID NeuroMat produziu {num_articles} artigos e foram citados pelo menos {num_citations} vezes. Como ilustrado abaixo, a produção dependeu de uma rede ampla de co-autores, com {num_authors} pessoas. A cor nos nós do grafo representa o tipo de afiliação do autor, verde quando possui pelo menos uma conexão brasileira e laranja se possui apenas afiliações internacionais. No geral, esses números oferecem um vislumbre valioso das redes colaborativas de autores e partes interessadas que participam do trabalho do NeuroMat. <a href='https://query-scholarly.wikidata.org/#{urllib.parse.quote(query_6)}' target='_blank'>Experimente a consulta completa!</a>"""
paragrafo_6 = f"""As três principais revistas em que a equipe do NeuroMat publicou são: <i>{journal_1}</i>, <i>{journal_2}</i> e <i>{journal_3}</i>, respectivamente com {num_articles_1}, {num_articles_2} e {num_articles_3} artigos. Essas revistas têm classificação <a href='https://www.wikidata.org/wiki/Q10355865' target='_blank'>Qualis Capes</a>, em ordem, {qualis_1}, {qualis_2} e {qualis_3}."""
paragrafo_7 = "No gráfico abaixo é possível visualizar a quantidade de artigos publicados em revistas difusoras das pesquisas por ano, desde o surgimento do CEPID NeuroMat, a partir da classificação B2 do Qualis Capes."
paragrafo_8 = "Para complementar a visualização do histograma acima, o gráfico abaixo apresenta os tipos de autoria."
paragrafo_9 = "A partir do <a href='https://www.wikidata.org/wiki/Q24575409'>Modelo Galves-Löcherbach</a> e da criação do CEPID, em 2013, a missão de inovação do NeuroMat tornou-se o desenvolvimento de ferramentas e aplicações baseadas no novo quadro conceitual para o entendimento do cérebro. Devido a isso, as pesquisas realizadas pelo CEPID NeuroMat foram utilizadas em diversas conferências, em áreas como probabilidade, estatística e neurobiologia, tais como {Q16}, {Q17} e {Q18}. Algumas das revistas são responsáveis por publicações do CEPID NeuroMat por mais de uma vez."
paragrafo_10 = f"""A figura seguinte ilustra a evolução das áreas de pesquisa e tópicos de pesquisa desenvolvidos pela NeuroMat desde 2013. Entre os tópicos mais frequentes nas publicações da NeuroMat estão {", ".join(keywords[:5])}e outros campos relacionados."""
paragrafo_11 = f"O CEPID NeuroMat promoveu uma rede multidisciplinar de pesquisas com colaborações internacionais, envolvendo pesquisadores em {num_institutions} instituições, em {num_countries} países. O centro contou com participantes dos seguintes países: {join_with_and(country_list)}. O mapa abaixo situa as instituições envolvidas na rede de pesquisa do NeuroMat."
paragrafo_12 = f"A rede de pesquisadores do NeuroMat envolveu matemáticos, probabilistas, neurobiólogos, cientistas computacionais, estatísticos e físicos, entre outros. São {num_main_researchers} pesquisadores principais, {num_associate_researchers} pesquisadores associados, {num_postdoc_researchers} pós-doutores, {num_doctor_researchers} doutores e {num_master_researchers} mestres. No histograma abaixo pode-se visualizar a quantidade de pesquisadores, de acordo com a quantidade de artigos produzidos, e o país em que estavam localizados."
paragrafo_13 = "Para complementar a visualização do histograma acima, o gráfico abaixo apresenta em detalhes a quantidade de artigos produzidos por cada pesquisador, de acordo com a sua localização."
paragrafo14 = f"Na rede de pesquisa do CEPID NeuroMat, houve {genders["feminino"]} pessoas que se declararam do gênero feminino e {genders["masculino"]} que se declararam do gênero masculino."
legenda = """<span style="background-color: #31005e; border: 1px solid #000000; text-align: center;">&nbsp;&nbsp;</span> <span style="color: #31005e; ">&gt;30 citações</span>
<span style="background-color: #1a009c; border: 1px solid #000000; text-align: center;">&nbsp;&nbsp;</span> <span style="color: #1a009c; ">&gt;20 citações</span>
<span style="background-color: #0587e3; border: 1px solid #000000; text-align: center;">&nbsp;&nbsp;</span> <span style="color: #0587e3; ">&gt;15 citações</span>
<span style="background-color: #05e3af; border: 1px solid #000000; text-align: center;">&nbsp;&nbsp;</span> <span style="color: #05e3af; ">&gt;10 citações</span>
<span style="background-color: #7fe305; border: 1px solid #000000; text-align: center;">&nbsp;&nbsp;</span> <span style="color: #7fe305; ">&gt;5 citações</span>
<span style="background-color: #e3dc05; border: 1px solid #000000; text-align: center;">&nbsp;&nbsp;</span> <span style="color: #e3dc05; ">&gt;1 citação</span>
<span style="background-color: #e37b05; border: 1px solid #000000; text-align: center;">&nbsp;&nbsp;</span> <span style="color: #e37b05; ">1 citação</span>
<span style="background-color: #e30505; border: 1px solid #000000; text-align: center;">&nbsp;&nbsp;</span> <span style="color: #e30505; ">0 citações</span>"""

In [ ]:
########################################################################################################################################################
# EXIBIÇÃO DOS PARÁGRAFOS E IFRAMES
########################################################################################################################################################
display(Image("https://neuromat.numec.prp.usp.br/static/img/neuromat.png", width=1000, height=1300))
display(HTML("<html><head><title>NeuroMat</title></head>"))
display(HTML("<h1 style='font-size: 1.5em'><b style='font-size: 2em;'>NeuroMat</b> (Centro de Pesquisa, Inovação e Difusão em Neuromatemática)</h1>"))
display(HTML(paragrafo_1))
display(HTML(paragrafo_2))
display(HTML(paragrafo_3))
display(iframe_citacoes)
display(HTML(legenda))
display(HTML(paragrafo_4))
display(HTML("<h2>Publicações</h2>"))
display(HTML(paragrafo_5))
display(iframe_affiliation)
display(HTML(paragrafo_6))
display(HTML(paragrafo_7))
display(iframe_qualis)
display(HTML(paragrafo_8))
display(iframe_qualis_by_authors)
display(HTML(paragrafo_9))
display(HTML(paragrafo_10))
display(HTML("<h2>Pesquisadores(as)</h2>"))
display(iframe_keywords)
display(HTML(paragrafo_11))
display(iframe_affiliation)
display(HTML(paragrafo_12))
display(iframe_publications)
display(HTML(paragrafo_13))
display(iframe_publications_by_author)
display(HTML(paragrafo14))

## Impacto social

In [ ]:
####
# código atualizado para a Vitrine NeuroMat
####

import requests
import pandas as pd

# Nome do arquivo CSV de entrada
csv_file = "df_altmetric.csv"

# Carregar o CSV, especificando o delimitador como ';'
df = pd.read_csv(csv_file, delimiter=';', on_bad_lines='skip')  # Added delimiter and error handling


doi_score = df['doi_score']
readers = df['readers_count']
tweeters = df['cited_by_tweeters_count']
rdts = df['cited_by_rdts']
news_count = df['cited_by_msm_count']
feeds = df['cited_by_feeds_count']
accounts = df['cited_by_accounts_count']
fbwalls = df['cited_by_fbwalls_count']
gplus = df['cited_by_gplus_count']
videos = df['cited_by_videos_count']
wikipedia = df['cited_by_wikipedia_count']

# Converter as colunas para o tipo numérico


for ind in readers.index:
    if len(str(doi_score[ind])) > 20:
        doi_score.iloc[ind] = 0.0
    if len(str(readers[ind])) > 20:
        readers.iloc[ind] = 0.0
    if len(str(tweeters[ind])) > 20:
        tweeters.iloc[ind] = 0.0
    if len(str(rdts[ind])) > 20:
        rdts.iloc[ind] = 0.0
    if len(str(news_count[ind])) > 20:
        news_count.iloc[ind] = 0.0
    if len(str(feeds[ind])) > 20:
        feeds.iloc[ind] = 0.0
    if len(str(accounts[ind])) > 20:
        accounts.iloc[ind] = 0.0
    if len(str(fbwalls[ind])) > 20:
        fbwalls.iloc[ind] = 0.0
    if len(str(gplus[ind])) > 20:
        gplus.iloc[ind] = 0.0
    if len(str(videos[ind])) > 20:
        videos.iloc[ind] = 0.0
    if len(str(wikipedia[ind])) > 20:
        wikipedia.iloc[ind] = 0.0

doi_score = doi_score.fillna(0.0)
doi_score = doi_score.astype(float)

readers = readers.fillna(0.0)
readers = readers.astype(float)

tweeters = tweeters.fillna(0.0)
tweeters = tweeters.astype(float)

rdts = rdts.fillna(0.0)
rdts = rdts.astype(float)

news_count = news_count.fillna(0.0)
news_count = news_count.astype(float)

feeds = feeds.fillna(0.0)
feeds = feeds.astype(float)

accounts = accounts.fillna(0.0)
accounts = accounts.astype(float)

fbwalls = fbwalls.fillna(0.0)
fbwalls = fbwalls.astype(float)

gplus = gplus.fillna(0.0)
gplus = gplus.astype(float)

videos = videos.fillna(0.0)
videos = videos.astype(float)

wikipedia = wikipedia.fillna(0.0)
wikipedia = wikipedia.astype(float)

summary_intro = "As métricas alternativas, do inglês altmetrics, são definidas como o indicador de medidas de impacto de uma pesquisa, utilizando como base a atividade online. "

summary_score = "Até hoje, as pesquisas realizadas pela rede de pesquisadores do CEPID NeuroMat possuem o Altmetric Score total de {:.2f}. "
summary_score = summary_score.format(doi_score.sum())


summary_readers = "Este score é composto por {} leitores - através das plataformas Mendeley, CiteULike e Connotea, ".format(int(readers.sum()))

summary_mentions = "{} menções no Twitter, {} menções no Reddit, {} menções em sites de notícias, ".format(
    int(tweeters.sum()), int(rdts.sum()), int(news_count.sum()))

summary_blogs = "{} menções em blogs (acadêmicos e gerais), ".format(int(feeds.sum()))

summary_profiles = "{} diferentes perfis de fonte de dados, {} menções em páginas do Facebook, ".format(
    int(accounts.sum()), int(fbwalls.sum()))

summary_other = "{} menções no Google+, {} menções em sites de upload de vídeos. ".format(
    int(gplus.sum()), int(videos.sum()))

summary_wikipedia = "Há também {} artigos na Wikipédia que fazem uso da produção científica do NeuroMat e {} menções em sites de upload de vídeos. ".format(
    int(wikipedia.sum()), int(wikipedia.sum()))

altmetrics_summary = " ".join([
    summary_intro, summary_score, summary_readers, summary_mentions,
    summary_blogs, summary_profiles, summary_other, summary_wikipedia	
])

print(altmetrics_summary)


In [ ]:
#Etapa 2
#ordena os valores encontrados em cada váriavel e estrutura um parágrafo

import pandas as pd

# Nome do arquivo CSV gerado anteriormente
csv_file = "df_altmetric.csv"

# Carregar o CSV com delimitador ";"
df = pd.read_csv(csv_file, delimiter=";")

# Lista de colunas numéricas para análise
colunas_numericas = [
    "doi_score",
    "readers_count",
    "cited_by_tweeters_count",
    "cited_by_accounts_count",
    "cited_by_posts_count",
    "cited_by_msm_count",
    "cited_by_feeds_count",
    "cited_by_fbwalls_count",
    "cited_by_gplus_count",
    "cited_by_videos_count",
    "cited_by_rdts",
    "cited_by_wikipedia_count"
]

# Mapeamento de nomes de colunas para descrições amigáveis
descricoes_amigaveis = {
    "doi_score": "Altmetric Attention Score",
    "readers_count": "Número de Leitores",
    "cited_by_tweeters_count": "Menções no Twitter",
    "cited_by_accounts_count": "Menções por contas únicas",
    "cited_by_posts_count": "menções em postagens",
    "cited_by_msm_count": "menções na mídia tradicional",
    "cited_by_feeds_count": "menções em feeds de notícias",
    "cited_by_fbwalls_count": "menções no Facebook",
    "cited_by_gplus_count": "Menções no Google+",
    "cited_by_videos_count": "Menções em Vídeos",
    "cited_by_rdts": "Menções no Reddit",
    "cited_by_wikipedia_count": "Referências em páginas da Wikipédia"
}

# Garantir que os valores sejam numéricos, convertendo erros para NaN
df[colunas_numericas] = df[colunas_numericas].apply(pd.to_numeric, errors='coerce')

# Verificar se existe uma coluna de título no CSV
coluna_titulo = "título" if "título" in df.columns else "doi"

# Dicionário para armazenar os maiores valores
maiores_valores = {}

for coluna in colunas_numericas:
    if df[coluna].notna().any():  # Verifica se há valores numéricos válidos na coluna
        max_valor = df[coluna].max()
        artigo = df.loc[df[coluna] == max_valor, coluna_titulo].values[0]  # Pega o primeiro artigo com esse valor
        maiores_valores[coluna] = (max_valor, artigo)

# Exibir os resultados de forma natural
for coluna, (valor, artigo) in maiores_valores.items():
    descricao = descricoes_amigaveis.get(coluna, coluna)  # Usa a descrição amigável, se disponível
    print(f"O maior valor encontrado para '{descricao}' é {valor}'.")


O maior valor encontrado para 'Altmetric Attention Score' é 1770.77'.
O maior valor encontrado para 'Número de Leitores' é 892'.
O maior valor encontrado para 'Menções no Twitter' é 327'.
O maior valor encontrado para 'Menções por contas únicas' é 540'.
O maior valor encontrado para 'menções em postagens' é 641'.
O maior valor encontrado para 'menções na mídia tradicional' é 182'.
O maior valor encontrado para 'menções em feeds de notícias' é 20'.
O maior valor encontrado para 'menções no Facebook' é 19'.
O maior valor encontrado para 'Menções no Google+' é 11'.
O maior valor encontrado para 'Menções em Vídeos' é 3'.
O maior valor encontrado para 'Menções no Reddit' é 0'.
O maior valor encontrado para 'Referências na Wikipédia' é 7'.

In [ ]:
#Etapa 3
# Estrutura de narrativa (GLN) para o artigo DOI 10.1007/S10955-013-0733-9 -
# Sistemas Infinitos de Cadeias em Interação com Memória de Alcance
# Variável—Um Modelo Estocástico para Redes Neurais Biológicas

import pandas as pd
import requests
import json

''''
# Nome do arquivo CSV
csv_file = "df_altmetric.csv"

# Carregar o CSV com delimitador ";"
df = pd.read_csv(csv_file, delimiter=";")

# Garantir que os valores da coluna 'doi_score' sejam numéricos, convertendo erros para NaN
df["doi_score"] = pd.to_numeric(df["doi_score"], errors="coerce")

# Encontrar o maior valor na coluna 'doi_score'
maior_doi_score = df["doi_score"].max()

# Encontrar o artigo com o maior 'doi_score'
artigo_maior_score = df.loc[df["doi_score"] == maior_doi_score]

# Recuperar o DOI do artigo com o maior 'doi_score'
doi_maior_score = artigo_maior_score["doi"].values[0]


# Acessar a API do Altmetric com o DOI
url_api = f"https://api.altmetric.com/v1/doi/{doi_maior_score}"
response = requests.get(url_api)'
'''
# Acessar a API do Altmetric com o DOI
url_api = f"https://api.altmetric.com/v1/doi/10.1007/S10955-013-0733-9"
response = requests.get(url_api)
# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    dados_api = response.json()

    # Extrair informações relevantes do JSON
    titulo = dados_api.get("title", "Título não disponível")
    autores = dados_api.get("authors", [])
    tipo_producao = dados_api.get("type", "Tipo não disponível")
    local_publicacao = dados_api.get("journal", "Local de publicação não disponível")
    score = dados_api.get("score", "Score não disponível")
    posts = dados_api.get("cited_by_posts_count", "Menções em postagens não disponíveis")
    accounts = cited_by_accounts_count = dados_api.get("cited_by_accounts_count", "Menções em contas não disponíveis")
    wikipedia = cited_by_wikipedia_count = dados_api.get("cited_by_wikipedia_count", "Menções em wikis não disponíveis")

    # Estruturar a narrativa
    narrativa = (
        f"O artigo '{titulo}', publicado no {local_publicacao}, é a produção científica do tipo {tipo_producao.lower()} de maior impacto social no NeuroMat. "
        f"Com um Altmetric Attention Score de {score}, este trabalho foi produzido por {', '.join(autores) if autores else 'autores não disponíveis'}, "
        f"se destaca como uma produção do tipo {tipo_producao.lower()}. "
        f"O artigo possui {posts} citações em posts em blogs, em {accounts} contas distintas e com {wikipedia} páginas na Wikipédia que citam este artigo."

    )

    print(narrativa)
else:
    print(f"Erro ao acessar a API do Altmetric para este DOI. Código de status: {response.status_code}")

**Destaque altmétrico no NeuroMat**

O artigo **'Infinite Systems of Interacting Chains with Memory of Variable Length—A Stochastic Model for Biological Neural Nets'**, publicado no **Journal of Statistical Physics**, é a produção científica do tipo article de maior impacto social no NeuroMat. Com um Altmetric Attention Score de **3**, este trabalho foi produzido por **A. Galves, E. Löcherbach**. O artigo possui **7** citações em posts em blogs, em **7** contas distintas e com **7** páginas na Wikipédia que citam esse artigo.


In [ ]:
paragrafoAltmetrics6 = "O CEPID NeuroMat tem se empenhado em enviar arquivos de mídia para o repositório aberto Wikimedia Commons, o depósito de imagens e vídeos que alimenta a Wikipedia. "

QA15 = "43.1K"
paragrafoAltmetrics71 = "Até hoje, a iniciativa já contribuiu com %s arquivos de mídia para o Wikimedia Commons. "
paragrafoAltmetrics7 = paragrafoAltmetrics71 % (QA15) 
paragrafoAltmetrics67 = paragrafoAltmetrics6 + paragrafoAltmetrics7 

display(HTML("<font size='+0.8'>" + paragrafoAltmetrics67 + "</font>"))